# Import libraries

In [10]:
import os
import math
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# Loading the data

In [107]:
def load_data(dir_path, ID):
    df_train = pd.read_csv(f"{dir_path}/train_FD00{ID}.txt", sep=" ", header=None)
    df_test = pd.read_csv(f"{dir_path}/test_FD00{ID}.txt", sep=" ", header=None)
    df_rul = pd.read_csv(f"{dir_path}/RUL_FD00{ID}.txt", sep=" ", header=None)
    return df_train, df_test, df_rul

In [109]:
dir_path = "../../datasets/CMAPSS_JetEngine"
ID = 1
df_train, df_test, df_rul = load_data(dir_path, ID)

In [13]:
df_train.describe()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
count,20631.000000,20631.000000,20631.000000,20631.000000,20631.0,20631.00,20631.000000,20631.000000,20631.000000,2.063100e+04,...,20631.000000,20631.000000,2.063100e+04,20631.000000,20631.0,20631.0,20631.000000,20631.000000,0.0,0.0
mean,51.506568,108.807862,-0.000009,0.000002,100.0,518.67,642.680934,1590.523119,1408.933782,1.462000e+01,...,8143.752722,8.442146,3.000000e-02,393.210654,2388.0,100.0,38.816271,23.289705,NaN,NaN
std,29.227633,68.880990,0.002187,0.000293,0.0,0.00,0.500053,6.131150,9.000605,1.776400e-15,...,19.076176,0.037505,1.387812e-17,1.548763,0.0,0.0,0.180746,0.108251,NaN,NaN
min,1.000000,1.000000,-0.008700,-0.000600,100.0,518.67,641.210000,1571.040000,1382.250000,1.462000e+01,...,8099.940000,8.324900,3.000000e-02,388.000000,2388.0,100.0,38.140000,22.894200,NaN,NaN
25%,26.000000,52.000000,-0.001500,-0.000200,100.0,518.67,642.325000,1586.260000,1402.360000,1.462000e+01,...,8133.245000,8.414900,3.000000e-02,392.000000,2388.0,100.0,38.700000,23.221800,NaN,NaN
50%,52.000000,104.000000,0.000000,0.000000,100.0,518.67,642.640000,1590.100000,1408.040000,1.462000e+01,...,8140.540000,8.438900,3.000000e-02,393.000000,2388.0,100.0,38.830000,23.297900,NaN,NaN
75%,77.000000,156.000000,0.001500,0.000300,100.0,518.67,643.000000,1594.380000,1414.555000,1.462000e+01,...,8148.310000,8.465600,3.000000e-02,394.000000,2388.0,100.0,38.950000,23.366800,NaN,NaN
max,100.000000,362.000000,0.008700,0.000600,100.0,518.67,644.530000,1616.910000,1441.490000,1.462000e+01,...,8293.720000,8.584800,3.000000e-02,400.000000,2388.0,100.0,39.430000,23.618400,NaN,NaN


In [14]:
# Remove the columns 26, 27 because all rows are "None" values
df_train.drop(columns=[26, 27], inplace=True)
df_test.drop(columns=[26, 27], inplace=True)

In [15]:
# The detailed dataset description can be found in the paper "Damage Propagation Modeling for Aircraft Engine Run-to-Failure Simulation".
name_columns = ['unit_number','time_in_cycles','setting_1','setting_2','TRA','T2','T24','T30','T50','P2','P15','P30','Nf', 
                'Nc','epr','Ps30','phi','NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32' ]

In [16]:
df_train.columns = name_columns
df_test.columns = name_columns

In [17]:
df_test.head()

,unit_number,time_in_cycles,setting_1,setting_2,TRA,T2,T24,T30,T50,P2,...,phi,NRf,NRc,BPR,farB,htBleed,Nf_dmd,PCNfR_dmd,W31,W32
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [18]:
# Some types of measurement data are constant values, which cannot provide valuable information for the RUL prediction.
# Remove these columns ['Nf_dmd','PCNfR_dmd','P2','T2','TRA','farB','epr']

df_train.drop(columns=['Nf_dmd','PCNfR_dmd','P2','T2','TRA','farB','epr'], inplace=True)
df_test.drop(columns=['Nf_dmd','PCNfR_dmd','P2','T2','TRA','farB','epr'], inplace=True)

In [19]:
def process_data(data):
    df = data.copy()
    pd_RUL = df.groupby('unit_number')['time_in_cycles'].max().reset_index()
    pd_RUL.columns = ['unit_number', 'max']
    df = df.merge(pd_RUL, on=['unit_number'], how='left')
    df['RUL'] = df['max'] - df['time_in_cycles']
    df.drop(columns=['max'], inplace=True)
    
    return df[df['time_in_cycles'] > 0]

In [99]:
X_train = process_data(df_train)
X_test = process_data(df_test)
X_test.shape

(13096, 20)

# Visualize the correlation matrix data

In [21]:
sns.heatmap(df.corr(), annot=True, cmap='RdYlGn',linewidths=0.2)
fig=plt.gcf()
fig.set_size_inches(18,10)
plt.show()

NameError: name 'sns' is not defined

# Evaluation Metrics

In [22]:
# Equation (21)
def calculate_s_score(y_ground_truth, y_predicted_value, positive_exp_factor=10, negative_exp_factor=13):
    s_score = 0
    losses = y_predicted_value - y_ground_truth
    for loss in losses:
        if loss >= 0:
            s_score += math.exp(loss/positive_exp_factor) - 1
        else:
            s_score += math.exp(-loss/negative_exp_factor) - 1
    return s_score

In [23]:
def evaluate_model(y_ground_truth, y_predicted_value):
    """
    Return RMSE and s_score
    """
    rmse = mean_squared_error(y_ground_truth, y_predicted_value, squared=False)
    s_score = calculate_s_score(y_ground_truth, y_predicted_value)
    return rmse, s_score

# LSTM (Long Short-Term Memory)

## Data Preprocessing Functions

In [24]:
def normalize_data(df, cols_to_normalize):
    min_max_scaler = MinMaxScaler()
    df[cols_to_normalize] = min_max_scaler.fit_transform(df[cols_to_normalize])
    return df

In [25]:
WINDOW_LENGTH = 30

def generate_sequences(df, seq_length):
    pass

In [26]:
def preprocess_data(train_data, test_data, rul_data):
    # Makes copies of the raw data
    train_df = train_data.copy()
    test_df = test_data.copy()
    rul_df = rul_data.copy()
    
    # Drop unnecessary columns from the RUL data
    rul_df.drop(columns=rul_df.columns[1], axis=1, inplace=True)
    
    # Normalize the training data
    cols_to_normalize = train_df.columns.difference(['unit_number', 'time_in_cycles', 'RUL'])
    train_df = normalize_data(train_df, cols_to_normalize)
    
    
    # Normalize the testing data
    test_df = normalize_data(test_df, cols_to_normalize)

In [63]:
X_train.columns

Index(['unit_number', 'time_in_cycles', 'setting_1', 'setting_2', 'T24', 'T30',
       'T50', 'P15', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR',
       'htBleed', 'W31', 'W32', 'RUL'],
      dtype='object')

In [50]:
data_train = X_train.iloc[:, :19].to_numpy()
label_train = X_train.iloc[:, 19:].to_numpy()
label_train = np.ravel(label_train)
label_train

array([191, 190, 189, ...,   2,   1,   0], dtype=int64)

In [52]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=70, max_features=7, max_depth=5, n_jobs=-1, random_state=1)
model.fit(data_train, label_train)

RandomForestRegressor(max_depth=5, max_features=7, n_estimators=70, n_jobs=-1,
                      random_state=1)

In [100]:
test_max = X_test.groupby(['unit_number'])['time_in_cycles'].max().reset_index()
test_max.columns = ['unit_number', 'max']

In [101]:
X_test = X_test.merge(test_max, on=['unit_number'], how='left')
X_test.head()

,unit_number,time_in_cycles,setting_1,setting_2,T24,T30,T50,P15,P30,Nf,...,Ps30,phi,NRf,NRc,BPR,htBleed,W31,W32,RUL,max
0,1,1,0.0023,0.0003,643.02,1585.29,1398.21,21.61,553.90,2388.04,...,47.20,521.72,2388.03,8125.55,8.4052,392,38.86,23.3735,30,31
1,1,2,-0.0027,-0.0003,641.71,1588.45,1395.42,21.61,554.85,2388.01,...,47.50,522.16,2388.06,8139.62,8.3803,393,39.02,23.3916,29,31
2,1,3,0.0003,0.0001,642.46,1586.94,1401.34,21.61,554.11,2388.05,...,47.50,521.97,2388.03,8130.10,8.4441,393,39.08,23.4166,28,31
3,1,4,0.0042,0.0000,642.44,1584.12,1406.42,21.61,554.07,2388.03,...,47.28,521.38,2388.05,8132.90,8.3917,391,39.00,23.3737,27,31
4,1,5,0.0014,0.0000,642.51,1587.19,1401.92,21.61,554.16,2388.01,...,47.31,522.15,2388.03,8129.54,8.4031,390,38.99,23.4130,26,31


In [102]:
test = X_test[X_test['time_in_cycles'] == X_test['max']].reset_index()

In [103]:
test.drop(columns=['index', 'max', 'RUL'], inplace=True)

In [104]:
test

,unit_number,time_in_cycles,setting_1,setting_2,T24,T30,T50,P15,P30,Nf,Nc,Ps30,phi,NRf,NRc,BPR,htBleed,W31,W32
0,1,31,-0.0006,0.0004,642.58,1581.22,1398.91,21.61,554.42,2388.08,9056.40,47.23,521.79,2388.06,8130.11,8.4024,393,38.81,23.3552
1,2,49,0.0018,-0.0001,642.55,1586.59,1410.83,21.61,553.52,2388.10,9044.77,47.67,521.74,2388.09,8126.90,8.4505,391,38.81,23.2618
2,3,126,-0.0016,0.0004,642.88,1589.75,1418.89,21.61,552.59,2388.16,9049.26,47.88,520.83,2388.14,8131.46,8.4119,395,38.93,23.2740
3,4,106,0.0012,0.0004,642.78,1594.53,1406.88,21.61,552.64,2388.13,9051.30,47.65,521.88,2388.11,8133.64,8.4634,395,38.58,23.2581
4,5,98,-0.0013,-0.0004,642.27,1589.94,1419.36,21.61,553.29,2388.10,9053.99,47.46,521.00,2388.15,8125.74,8.4362,394,38.75,23.4117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,97,-0.0006,0.0003,642.30,1590.88,1397.94,21.61,553.99,2388.03,9062.41,47.14,522.30,2388.01,8148.24,8.4110,391,38.96,23.4606
96,97,134,0.0013,-0.0001,642.59,1582.96,1410.92,21.61,554.05,2388.06,9076.36,47.38,521.58,2388.06,8155.48,8.4500,395,38.61,23.2953
97,98,121,0.0017,0.0001,642.68,1599.51,1415.47,21.61,553.44,2388.13,9062.34,47.66,521.53,2388.09,8146.39,8.4235,394,38.76,23.3608
98,99,97,0.0047,-0.0000,642.00,1585.03,1397.98,21.61,554.75,2388.01,9067.16,47.26,521.82,2388.02,8150.38,8.4003,391,38.95,23.3595


In [113]:
predicted_values = model.predict(test)

C:\Users\HTrung\.conda\envs\hieuvt\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [112]:
y_true = df_rul[0].to_numpy()

In [114]:
evaluate_model(y_true, predicted_values)

(24.995312064930886, 4791.693663636706)